# Quickstart

https://www.django-rest-framework.org/tutorial/quickstart/

## Project setup
- startproject is run with a . after the project name to put the app in the project directory, but it helps avoid clashes with external module's namespaces

## Serializers
`HyperlinkedModelSerializer` used instead of primary key and various other relationships - hyperlinking is good RESTful design

## Views
We group common behavior into classes called `ViewSets`.  These can be very concise and organized.

## URLs
When using viewsets, we can generate the URL conf for our API by registering viewsets with a router class in `urls.py`
- if we need more control over API URLs we can use class-based views and explicitely write URL conf
- default login and logout views are added for browsable API

## Pagination
Determines how many objects per page are returned.  Edit `settings.py`.

# API access
Run the server, then 
`curl -H 'Accept: application/json; indent=4' -u admin:password123 http://127.0.0.1:8000/users/`
or with HTTPie
`http -a admin:password123 http://127.0.0.1:8000/users/`
or point the browser to
http://127.0.0.1:8000/users/

# In-depth Tutorial
https://www.django-rest-framework.org/tutorial/1-serialization/

The tutorial walks through setting up the snippets app, and creating a model for it.

## Creating a Serializer class
We can serialize data into `json` by declaring serializers in a way similar to Django's forms

### snippets/serializers.py

In [ ]:
from rest_framework import serializers
from snippets.models import Snippet, LANGUAGE_CHOICES, STYLE_CHOICES


class SnippetSerializer(serializers.Serializer):
    id = serializers.IntegerField(read_only=True)
    title = serializers.CharField(required=False, allow_blank=True, max_length=100)
    code = serializers.CharField(style={'base_template': 'textarea.html'})
    linenos = serializers.BooleanField(required=False)
    language = serializers.ChoiceField(choices=LANGUAGE_CHOICES, default='python')
    style = serializers.ChoiceField(choices=STYLE_CHOICES, default='friendly')

    def create(self, validated_data):
        """
        Create and return a new `Snippet` instance, given the validated data.
        """
        return Snippet.objects.create(**validated_data)

    def update(self, instance, validated_data):
        """
        Update and return an existing `Snippet` instance, given the validated data.
        """
        instance.title = validated_data.get('title', instance.title)
        instance.code = validated_data.get('code', instance.code)
        instance.linenos = validated_data.get('linenos', instance.linenos)
        instance.language = validated_data.get('language', instance.language)
        instance.style = validated_data.get('style', instance.style)
        instance.save()
        return instance

### Serializing

Now that we have this set up, run `python manage.py shell` from the terminal.  Once inside:

In [ ]:
from snippets.models import Snippet
from snippets.serializers import SnippetSerializer
from rest_framework.renderers import JSONRenderer
from rest_framework.parsers import JSONParser

# we'll add some data to play with
snippet = Snippet(code='foo = "bar"\n')
snippet.save()

snippet = Snippet(code='print("hello, world")\n')
snippet.save()

In [ ]:
# now we can see what the serialized output is
# this will be Python native datatypes
serializer = SnippetSerializer(snippet)
serializer.data

In [ ]:
# render to json
content = JSONRenderer().render(serializer.data)
content

### Deserialization

In [ ]:
import io

# parse a stream into Python native datatypes
stream = io.BytesIO(content)
data = JSONParser().parse(stream)

In [ ]:
# restore native datatypes to object instance
serializer = SnippetSerializer(data=data)
serializer.is_valid()
# True
serializer.validated_data
# OrderedDict([('title', ''), ('code', 'print("hello, world")\n'), ('linenos', False), ('language', 'python'), ('style', 'friendly')])
serializer.save()
# <Snippet: Snippet object>

### Serializing querysets
Add `many=True` to serializer arguments

In [ ]:
serializer = SnippetSerializer(Snippet.objects.all(), many=True)
serializer.data
# [OrderedDict([('id', 1), ('title', ''), ('code', 'foo = "bar"\n'), ('linenos', False), ('language', 'python'), ('style', 'friendly')]), OrderedDict([('id', 2), ('title', ''), ('code', 'print("hello, world")\n'), ('linenos', False), ('language', 'python'), ('style', 'friendly')]), OrderedDict([('id', 3), ('title', ''), ('code', 'print("hello, world")'), ('linenos', False), ('language', 'python'), ('style', 'friendly')])]

### ModelSerializers
`SnippetSerializer` and the `Snippet` model are a little redundant.  REST provides `Serializer` and `ModelSerializer` classes.  We can rewrite `snippets/serializers` by replacing `SnippetsSerializer`

In [ ]:
class SnippetSerializer(serializers.ModelSerializer):
    class Meta:
        model = Snippet
        fields = ['id', 'title', 'code', 'linenos', 'language', 'style']